In [1]:
import pandas as pd
import numpy as np
import gspread
from datetime import date
from datetime import datetime
from datetime import datetime, timedelta
from pandas import json_normalize

import re

import sys                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
sys.path.append('../')

import zoho_crm_api_module as crm

import sys
sys.path.append('../../../zoho_data_app/')

# importing
import main_module as m

%load_ext autoreload
%autoreload 2

In [2]:
gc = gspread.oauth(
    credentials_filename= r'C:\Users\fajar.fatoni\Documents\Python\Data\Google Credentials\karma-oauth.json'
)

SHEET_ID = '1_zOrWK1gAo-tHDsKqU6AaqVzi4xaNpOELDwtbBx5F6M'
SHEET_NAME = 'Karma Beach - LIVE'


cols = [
    "id","Created_Time", "First_Name", "Last_Name", "Email", "Birthdate", "Phone", 
    "Street", "City", "Country", "Brand", "Lead_Sub_Brand", "Lead_Source", 
    "Lead_Source_Description", "Lead_Locations", "Lead_History", "Website", 
    "Card_Number", "Card_Expired", "Do_you_live_in_Bali"
]

live_cols = [
    "Record Id", "Created Time", "First Name", "Last Name", "Email", "Birthdate", 
    "Phone", "Street", "City", "Country", "Lead Brand", "Lead Sub-Brand", 
    "Lead Source", "Lead Source Description", "Lead Locations", "Lead History", 
    "Website", "Card Number", "Card Expired", "Do you live in Bali?"
]

# Function to fix the encoding
def fix_encoding(garbled_string):
    try:
        # Encode the garbled string to bytes using 'latin1' and then decode it to 'utf-8'
        return garbled_string.encode('latin1').decode('utf-8')
    except Exception as e:
        return garbled_string  # Return the original string if there's an error


In [3]:
crm_token = crm.get_crm_token()

In [4]:
df = crm.get_kbb_vip_data(crm_token, cols)
df.fillna('empty', inplace=True)
new_df = crm.get_kbb_vip_filter(df)
new_df['id'] = 'zcrm_'+new_df['id']
new_df.rename(columns={'id': 'Record Id'}, inplace=True)
new_df['Created_Time'] = new_df['Created_Time'].str.replace('T', ' ').str.replace(r'\+\d{2}:\d{2}', '', regex=True)
new_df['Created_Time']  = pd.to_datetime(new_df['Created_Time']).dt.strftime('%d/%m/%Y')
new_df['Card_Expired']  = pd.to_datetime(new_df['Card_Expired']).dt.strftime('%d/%m/%Y')
new_df['First_Name'] = new_df['First_Name'].apply(fix_encoding)
new_df['Last_Name'] = new_df['Last_Name'].apply(fix_encoding)
m.clean_number(new_df, 'Phone')
m.replace_not_valid_to_empty(new_df, 'Phone')
# new_df = df.copy()
new_df.sort_values('Created_Time', ascending=False, inplace=True)
new_df.reset_index(drop=True, inplace=True)
count_data_awal = new_df.shape[0]
new_df

c:\Users\fajar.fatoni\Documents\Python\Task Offline\zoho_data_app\zoho_api\sync_module\..\zoho_crm_api_module.py:267: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  concat_df['Lead_Locations'].replace('\[|\]|\'','', regex=True, inplace=True)
c:\Users\fajar.fatoni\Documents\Python\Task Offline\zoho_data_app\zoho_api\sync_module\../../../zoho_data_app\main_module.py:104: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pan

,Record Id,Created_Time,First_Name,Last_Name,Email,Birthdate,Phone,Street,City,Country,Brand,Lead_Sub_Brand,Lead_Source,Lead_Source_Description,Lead_Locations,Lead_History,Website,Card_Number,Card_Expired,Do_you_live_in_Bali
0,zcrm_3584690000559215075,24/06/2025,Zackarias,Piehl,zackariaspiehl@gmail.com,empty,46709269315,empty,empty,Sweden,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000600240001,24/06/2026,False
1,zcrm_3584690000559213033,23/06/2025,Luana,Macurdy,nana.macurdy@gmail.com,empty,33750634806,empty,empty,France,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000600230006,23/06/2026,False
2,zcrm_3584690000559173147,23/06/2025,Barreau,Aurelien,alfaorel9@gmail.com,empty,33750634806,empty,empty,France,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000600230007,23/06/2026,False
3,zcrm_3584690000559150250,23/06/2025,Christy,Miranda,Admin@sortedinstyle.com,empty,61418223545,empty,empty,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000600230005,23/06/2026,False
4,zcrm_3584690000559107071,23/06/2025,Alexander,Führer,alex.alya2609@gmail.com,empty,491629553565,empty,empty,Germany,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000600230002,23/06/2026,False
5,zcrm_3584690000559075110,23/06/2025,Alya,Anindhita,alya.anindhita25@gmail.com,empty,491629553565,empty,empty,Germany,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000600230001,23/06/2026,False
6,zcrm_3584690000559057186,23/06/2025,Handi,Widjaja,widjaja.handic@gmail.com,empty,628119102498,empty,empty,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000600230004,23/06/2026,False
7,zcrm_3584690000559036181,23/06/2025,Monica,Widjaja,monicakumala@gmail.com,empty,628119127702,empty,empty,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000600230003,23/06/2026,False
8,zcrm_3584690000559094087,22/06/2025,Jacqueline,Simpkin,jackiesimpkin@live.com,empty,61427985404,empty,empty,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000600220001,22/06/2026,False
9,zcrm_3584690000559071001,21/06/2025,Abdul,Chami,abdchami9090@gmail.com,empty,966583857555,empty,empty,Saudi Arabia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000600210001,21/06/2026,False


In [5]:
new_df['First_Name'] = new_df['First_Name'].str.title()
new_df['Last_Name'] = new_df['Last_Name'].str.title()
new_df['Email'] = m.lowercase(new_df, 'Email')
m.clean_space(new_df, 'Email')

new_df
# new_df.to_excel(r"C:\Users\fajar.fatoni\Documents\Python\Data\kbbvip_live.xlsx", index=False)

c:\Users\fajar.fatoni\Documents\Python\Task Offline\zoho_data_app\zoho_api\sync_module\../../../zoho_data_app\main_module.py:127: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[df_columns].replace(' ','', regex=True, inplace=True)


,Record Id,Created_Time,First_Name,Last_Name,Email,Birthdate,Phone,Street,City,Country,Brand,Lead_Sub_Brand,Lead_Source,Lead_Source_Description,Lead_Locations,Lead_History,Website,Card_Number,Card_Expired,Do_you_live_in_Bali
0,zcrm_3584690000559215075,24/06/2025,Zackarias,Piehl,zackariaspiehl@gmail.com,empty,46709269315,empty,empty,Sweden,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000600240001,24/06/2026,False
1,zcrm_3584690000559213033,23/06/2025,Luana,Macurdy,nana.macurdy@gmail.com,empty,33750634806,empty,empty,France,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000600230006,23/06/2026,False
2,zcrm_3584690000559173147,23/06/2025,Barreau,Aurelien,alfaorel9@gmail.com,empty,33750634806,empty,empty,France,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000600230007,23/06/2026,False
3,zcrm_3584690000559150250,23/06/2025,Christy,Miranda,admin@sortedinstyle.com,empty,61418223545,empty,empty,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000600230005,23/06/2026,False
4,zcrm_3584690000559107071,23/06/2025,Alexander,Führer,alex.alya2609@gmail.com,empty,491629553565,empty,empty,Germany,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000600230002,23/06/2026,False
5,zcrm_3584690000559075110,23/06/2025,Alya,Anindhita,alya.anindhita25@gmail.com,empty,491629553565,empty,empty,Germany,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000600230001,23/06/2026,False
6,zcrm_3584690000559057186,23/06/2025,Handi,Widjaja,widjaja.handic@gmail.com,empty,628119102498,empty,empty,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000600230004,23/06/2026,False
7,zcrm_3584690000559036181,23/06/2025,Monica,Widjaja,monicakumala@gmail.com,empty,628119127702,empty,empty,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000600230003,23/06/2026,False
8,zcrm_3584690000559094087,22/06/2025,Jacqueline,Simpkin,jackiesimpkin@live.com,empty,61427985404,empty,empty,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000600220001,22/06/2026,False
9,zcrm_3584690000559071001,21/06/2025,Abdul,Chami,abdchami9090@gmail.com,empty,966583857555,empty,empty,Saudi Arabia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000600210001,21/06/2026,False


In [6]:
# # mencari tanggal 8 hari kebelakang
# calculate_date = datetime.now() - timedelta(days=8)
# new_cal_date = str(calculate_date.strftime("%Y-%m-%d")).split(' ')
# first_date = new_cal_date[0]

# # mencari tanggal hari ini
# today = datetime.now() 
# second_date = today.strftime("%Y-%m-%d")

# print(first_date)
# print(second_date)

In [7]:
new_df.replace('empty', '', inplace=True)
new_df

,Record Id,Created_Time,First_Name,Last_Name,Email,Birthdate,Phone,Street,City,Country,Brand,Lead_Sub_Brand,Lead_Source,Lead_Source_Description,Lead_Locations,Lead_History,Website,Card_Number,Card_Expired,Do_you_live_in_Bali
0,zcrm_3584690000559215075,24/06/2025,Zackarias,Piehl,zackariaspiehl@gmail.com,,46709269315,,,Sweden,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000600240001,24/06/2026,False
1,zcrm_3584690000559213033,23/06/2025,Luana,Macurdy,nana.macurdy@gmail.com,,33750634806,,,France,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000600230006,23/06/2026,False
2,zcrm_3584690000559173147,23/06/2025,Barreau,Aurelien,alfaorel9@gmail.com,,33750634806,,,France,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000600230007,23/06/2026,False
3,zcrm_3584690000559150250,23/06/2025,Christy,Miranda,admin@sortedinstyle.com,,61418223545,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000600230005,23/06/2026,False
4,zcrm_3584690000559107071,23/06/2025,Alexander,Führer,alex.alya2609@gmail.com,,491629553565,,,Germany,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000600230002,23/06/2026,False
5,zcrm_3584690000559075110,23/06/2025,Alya,Anindhita,alya.anindhita25@gmail.com,,491629553565,,,Germany,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000600230001,23/06/2026,False
6,zcrm_3584690000559057186,23/06/2025,Handi,Widjaja,widjaja.handic@gmail.com,,628119102498,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000600230004,23/06/2026,False
7,zcrm_3584690000559036181,23/06/2025,Monica,Widjaja,monicakumala@gmail.com,,628119127702,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000600230003,23/06/2026,False
8,zcrm_3584690000559094087,22/06/2025,Jacqueline,Simpkin,jackiesimpkin@live.com,,61427985404,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000600220001,22/06/2026,False
9,zcrm_3584690000559071001,21/06/2025,Abdul,Chami,abdchami9090@gmail.com,,966583857555,,,Saudi Arabia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000600210001,21/06/2026,False


In [8]:
import pandas as pd
# Assuming gc, SHEET_ID, SHEET_NAME, and fix_encoding are defined

df_live = pd.DataFrame() # Initialize an empty DataFrame

try:
    # Get data from Google Sheets
    spreadsheet = gc.open_by_key(SHEET_ID)
    worksheet = spreadsheet.worksheet(SHEET_NAME)
    # get_all_records assumes the first row is a header
    rows = worksheet.get_all_records()

    # Check if any data was returned
    if not rows:
        print(f"No data found in sheet '{SHEET_NAME}' or it lacks a header row.")
        # Define expected columns for an empty DataFrame if needed downstream
        df_live = pd.DataFrame(columns=['Card Expired', 'First Name', 'Last Name'])
    else:
        # Create DataFrame from the rows
        df_live = pd.DataFrame(rows)

        # --- Process columns only if they exist ---

        # Process 'Card Expired'
        if 'Card Expired' in df_live.columns:
            # Convert 'Card Expired' to datetime, handling potential errors
            df_live['Card Expired'] = pd.to_datetime(
                df_live['Card Expired'],
                dayfirst=True,
                errors='coerce' # Convert unparseable dates to NaT (Not a Time)
            )
            # Format back to string, handling NaT values gracefully
            df_live['Card Expired'] = df_live['Card Expired'].dt.strftime('%d/%m/%Y').fillna('') # Replace NaT with empty string
        else:
            print("Warning: Column 'Card Expired' not found in the sheet.")

        # Process 'First Name'
        if 'First Name' in df_live.columns:
            df_live['First Name'] = df_live['First Name'].apply(fix_encoding)
        else:
            print("Warning: Column 'First Name' not found in the sheet.")

        # Process 'Last Name'
        if 'Last Name' in df_live.columns:
            df_live['Last Name'] = df_live['Last Name'].apply(fix_encoding)
        else:
            print("Warning: Column 'Last Name' not found in the sheet.")

except Exception as e:
    print(f"An error occurred while accessing or processing the Google Sheet: {e}")
    # Ensure df_live is an empty DataFrame in case of error during processing
    if not isinstance(df_live, pd.DataFrame) or df_live.empty:
       df_live = pd.DataFrame(columns=['Card Expired', 'First Name', 'Last Name']) # Or just pd.DataFrame()
    print("DataFrame might be empty or partially processed due to the error.")


# Display the DataFrame
print("\nFinal DataFrame contents:")
# Use display() in environments like Jupyter/Colab, otherwise use print()
try:
    display(df_live)
except NameError:
    print(df_live)



Final DataFrame contents:


,Record Id,Created Time,First Name,Last Name,Email,Birthdate,Phone,Street,City,Country,Lead Brand,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Locations,Lead History,Website,Card Number,Card Expired,Do you live in Bali?
0,zcrm_3584690000559107071,23/06/2025,Alexander,Führer,alex.alya2609@gmail.com,,491629553565,,,Germany,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000600230002,23/06/2026,FALSE
1,zcrm_3584690000559075110,23/06/2025,Alya,Anindhita,alya.anindhita25@gmail.com,,491629553565,,,Germany,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000600230001,23/06/2026,FALSE
2,zcrm_3584690000559057186,23/06/2025,Handi,Widjaja,widjaja.handic@gmail.com,,628119102498,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000600230004,23/06/2026,FALSE
3,zcrm_3584690000559036181,23/06/2025,Monica,Widjaja,monicakumala@gmail.com,,628119127702,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000600230003,23/06/2026,FALSE
4,zcrm_3584690000559094087,22/06/2025,Jacqueline,Simpkin,jackiesimpkin@live.com,,61427985404,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000600220001,22/06/2026,FALSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9886,zcrm_3584690000007840074,02/04/2019,Oreste,Lentidoro,lentidoro@gmail.com,,61410699788,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,Other - Other,https://karmagroup.com/karma-beach-vip/,2024000300100005,10/03/2025,FALSE
9887,zcrm_3584690000007849782,02/04/2019,Shierley,Koval,shierleykoval@gmail.com,,622153688631,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,Other - Other,https://karmagroup.com/karma-beach-vip/,2024000300080013,08/03/2025,FALSE
9888,zcrm_3584690000008003243,02/04/2019,Debby,Lumempouw,dmsl_della@yahoo.com,,6281806204001,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,Other - Other,https://karmagroup.com/karma-beach-vip/,2024000200190007,19/02/2025,FALSE
9889,zcrm_3584690000008059311,02/04/2019,Meylina,Ardiningsih,meylinaardi@gmail.com,,6281385951854,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,Other - Other,https://karmagroup.com/karma-beach-vip/,2024000100200010,20/01/2025,FALSE


In [9]:
df_update = new_df.loc[~new_df['Record Id'].isin(df_live['Record Id'])].copy()
df_update

,Record Id,Created_Time,First_Name,Last_Name,Email,Birthdate,Phone,Street,City,Country,Brand,Lead_Sub_Brand,Lead_Source,Lead_Source_Description,Lead_Locations,Lead_History,Website,Card_Number,Card_Expired,Do_you_live_in_Bali
0,zcrm_3584690000559215075,24/06/2025,Zackarias,Piehl,zackariaspiehl@gmail.com,,46709269315,,,Sweden,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000600240001,24/06/2026,False
1,zcrm_3584690000559213033,23/06/2025,Luana,Macurdy,nana.macurdy@gmail.com,,33750634806,,,France,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000600230006,23/06/2026,False
2,zcrm_3584690000559173147,23/06/2025,Barreau,Aurelien,alfaorel9@gmail.com,,33750634806,,,France,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000600230007,23/06/2026,False
3,zcrm_3584690000559150250,23/06/2025,Christy,Miranda,admin@sortedinstyle.com,,61418223545,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000600230005,23/06/2026,False


In [10]:
df_update.sort_values(by=['Created_Time'], inplace=True, ascending=False)
rename_mapping = dict(zip(cols, live_cols))
df_update = df_update.rename(columns=rename_mapping)
df_update.reset_index(drop=True, inplace=True)
df_update

count_df_update = df_update.shape[0]


In [11]:
df_live

,Record Id,Created Time,First Name,Last Name,Email,Birthdate,Phone,Street,City,Country,Lead Brand,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Locations,Lead History,Website,Card Number,Card Expired,Do you live in Bali?
0,zcrm_3584690000559107071,23/06/2025,Alexander,Führer,alex.alya2609@gmail.com,,491629553565,,,Germany,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000600230002,23/06/2026,FALSE
1,zcrm_3584690000559075110,23/06/2025,Alya,Anindhita,alya.anindhita25@gmail.com,,491629553565,,,Germany,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000600230001,23/06/2026,FALSE
2,zcrm_3584690000559057186,23/06/2025,Handi,Widjaja,widjaja.handic@gmail.com,,628119102498,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000600230004,23/06/2026,FALSE
3,zcrm_3584690000559036181,23/06/2025,Monica,Widjaja,monicakumala@gmail.com,,628119127702,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000600230003,23/06/2026,FALSE
4,zcrm_3584690000559094087,22/06/2025,Jacqueline,Simpkin,jackiesimpkin@live.com,,61427985404,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000600220001,22/06/2026,FALSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9886,zcrm_3584690000007840074,02/04/2019,Oreste,Lentidoro,lentidoro@gmail.com,,61410699788,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,Other - Other,https://karmagroup.com/karma-beach-vip/,2024000300100005,10/03/2025,FALSE
9887,zcrm_3584690000007849782,02/04/2019,Shierley,Koval,shierleykoval@gmail.com,,622153688631,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,Other - Other,https://karmagroup.com/karma-beach-vip/,2024000300080013,08/03/2025,FALSE
9888,zcrm_3584690000008003243,02/04/2019,Debby,Lumempouw,dmsl_della@yahoo.com,,6281806204001,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,Other - Other,https://karmagroup.com/karma-beach-vip/,2024000200190007,19/02/2025,FALSE
9889,zcrm_3584690000008059311,02/04/2019,Meylina,Ardiningsih,meylinaardi@gmail.com,,6281385951854,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,Other - Other,https://karmagroup.com/karma-beach-vip/,2024000100200010,20/01/2025,FALSE


In [12]:
# Concatenate DataFrames
df_final = pd.concat([df_update, df_live], ignore_index=True)

# Replace 'empty' with ''
df_final.replace('empty', '', inplace=True)

# Convert 'Do you live in Bali?' to uppercase
df_final['Do you live in Bali?'] = df_final['Do you live in Bali?'].astype(str).str.upper()

# Convert 'Created Time' to datetime format
df_final['Created Time'] = pd.to_datetime(df_final['Created Time'], dayfirst=True, errors='coerce')

# Sort by 'Created Time' in descending order
df_final.sort_values('Created Time', ascending=False, inplace=True)

# Format 'Created Time' back to the desired string format
df_final['Created Time'] = df_final['Created Time'].dt.strftime('%d/%m/%Y')

df_final

,Record Id,Created Time,First Name,Last Name,Email,Birthdate,Phone,Street,City,Country,Lead Brand,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Locations,Lead History,Website,Card Number,Card Expired,Do you live in Bali?
0,zcrm_3584690000559215075,24/06/2025,Zackarias,Piehl,zackariaspiehl@gmail.com,,46709269315,,,Sweden,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000600240001,24/06/2026,FALSE
5,zcrm_3584690000559075110,23/06/2025,Alya,Anindhita,alya.anindhita25@gmail.com,,491629553565,,,Germany,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000600230001,23/06/2026,FALSE
7,zcrm_3584690000559036181,23/06/2025,Monica,Widjaja,monicakumala@gmail.com,,628119127702,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000600230003,23/06/2026,FALSE
6,zcrm_3584690000559057186,23/06/2025,Handi,Widjaja,widjaja.handic@gmail.com,,628119102498,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000600230004,23/06/2026,FALSE
1,zcrm_3584690000559213033,23/06/2025,Luana,Macurdy,nana.macurdy@gmail.com,,33750634806,,,France,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000600230006,23/06/2026,FALSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9871,zcrm_3584690000007147790,02/04/2019,Naveen,Singhal,naveensinghal007@gmail.com,,919901134010,,Bangalore,India,Karma Group,Other,Web Enquiry,Karma Beach VIP membership sign up 2023,,"Other - Web Enquiry, Karma Experience IN - Fac...",https://karmagroup.com/karma-beach-vip/,2024000100100043,10/01/2025,FALSE
9870,zcrm_3584690000005974937,02/04/2019,Paula,mcgeown,paulamcgeown@gmail.com,,61481151026,,,Australia,Karma Resorts,Other,Past Guests,Karma Beach VIP membership sign up 2023,Karma Rottnest,"Other - Past Guests, BGAU - Prospect",https://karmagroup.com/karma-beach-vip/,2025000100170001,17/01/2026,FALSE
9869,zcrm_3584690000005990930,02/04/2019,Jordan,Powell,jordanpowell7@hotmail.com,,64225901649,,,New Zealand,Karma Resorts,Other,Other,Karma Beach VIP membership sign up 2023,Karma Rottnest,"Other - Past Guests, Karma Royal - Karma Royal...",https://karmagroup.com/karma-beach-vip/,2024000500240017,24/05/2025,FALSE
9868,zcrm_3584690000005948267,02/04/2019,Elycia,Martins,elycia.martins@gmail.com,,61403924069,,,Australia,Karma Resorts,Other,Past Guests,Karma Beach VIP membership sign up 2023,Karma Rottnest,Other - Past Guests,https://karmagroup.com/karma-beach-vip/,2024000900090001,09/09/2025,FALSE


In [13]:
df_live.columns
# new_df.columns

Index(['Record Id', 'Created Time', 'First Name', 'Last Name', 'Email',
       'Birthdate', 'Phone', 'Street', 'City', 'Country', 'Lead Brand',
       'Lead Sub-Brand', 'Lead Source', 'Lead Source Description',
       'Lead Locations', 'Lead History', 'Website', 'Card Number',
       'Card Expired', 'Do you live in Bali?'],
      dtype='object')

In [14]:
# Function to sanitize DataFrame
def sanitize_dataframe(df):
    # Replace invalid float values with None
    df = df.applymap(lambda x: None if isinstance(x, float) and (np.isinf(x) or np.isnan(x)) else x)
    return df

# Sanitize the DataFrame
df_final = sanitize_dataframe(df_final)
df_final['Card Number'] = df_final['Card Number'].astype(str)
df_final.drop(df_final.loc[df_final['Email'].str.contains('karmagroup.com', regex=True, flags=re.I)].index, inplace=True)
df_final

C:\Users\fajar.fatoni\AppData\Local\Temp\ipykernel_7612\4224994593.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: None if isinstance(x, float) and (np.isinf(x) or np.isnan(x)) else x)


,Record Id,Created Time,First Name,Last Name,Email,Birthdate,Phone,Street,City,Country,Lead Brand,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Locations,Lead History,Website,Card Number,Card Expired,Do you live in Bali?
0,zcrm_3584690000559215075,24/06/2025,Zackarias,Piehl,zackariaspiehl@gmail.com,,46709269315,,,Sweden,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000600240001,24/06/2026,FALSE
5,zcrm_3584690000559075110,23/06/2025,Alya,Anindhita,alya.anindhita25@gmail.com,,491629553565,,,Germany,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000600230001,23/06/2026,FALSE
7,zcrm_3584690000559036181,23/06/2025,Monica,Widjaja,monicakumala@gmail.com,,628119127702,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000600230003,23/06/2026,FALSE
6,zcrm_3584690000559057186,23/06/2025,Handi,Widjaja,widjaja.handic@gmail.com,,628119102498,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000600230004,23/06/2026,FALSE
1,zcrm_3584690000559213033,23/06/2025,Luana,Macurdy,nana.macurdy@gmail.com,,33750634806,,,France,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000600230006,23/06/2026,FALSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9871,zcrm_3584690000007147790,02/04/2019,Naveen,Singhal,naveensinghal007@gmail.com,,919901134010,,Bangalore,India,Karma Group,Other,Web Enquiry,Karma Beach VIP membership sign up 2023,,"Other - Web Enquiry, Karma Experience IN - Fac...",https://karmagroup.com/karma-beach-vip/,2024000100100043,10/01/2025,FALSE
9870,zcrm_3584690000005974937,02/04/2019,Paula,mcgeown,paulamcgeown@gmail.com,,61481151026,,,Australia,Karma Resorts,Other,Past Guests,Karma Beach VIP membership sign up 2023,Karma Rottnest,"Other - Past Guests, BGAU - Prospect",https://karmagroup.com/karma-beach-vip/,2025000100170001,17/01/2026,FALSE
9869,zcrm_3584690000005990930,02/04/2019,Jordan,Powell,jordanpowell7@hotmail.com,,64225901649,,,New Zealand,Karma Resorts,Other,Other,Karma Beach VIP membership sign up 2023,Karma Rottnest,"Other - Past Guests, Karma Royal - Karma Royal...",https://karmagroup.com/karma-beach-vip/,2024000500240017,24/05/2025,FALSE
9868,zcrm_3584690000005948267,02/04/2019,Elycia,Martins,elycia.martins@gmail.com,,61403924069,,,Australia,Karma Resorts,Other,Past Guests,Karma Beach VIP membership sign up 2023,Karma Rottnest,Other - Past Guests,https://karmagroup.com/karma-beach-vip/,2024000900090001,09/09/2025,FALSE


In [15]:
# Update keseluruhan data, delete data yang lama terlebih dahulu
# clear all row before update
worksheet.clear()
worksheet.update([df_final.columns.values.tolist()] + df_final.values.tolist())

{'spreadsheetId': '1_zOrWK1gAo-tHDsKqU6AaqVzi4xaNpOELDwtbBx5F6M',
 'updatedRange': "'Karma Beach - LIVE'!A1:T9896",
 'updatedRows': 9896,
 'updatedColumns': 20,
 'updatedCells': 197920}

In [16]:
print("Data update kbb vip : "+str(count_df_update))

Data update kbb vip : 4
